# Example for Detecting Single Object (on White Background) and Draw Bounding Box

## Requirement

In [ ]:
%pip install scikit-image

## Importing Required Modules

In [ ]:
import os
import time
import cv2 as cv
import numpy as np
from skimage import morphology as morph

## Specify Path for Input and Output File

In [ ]:
# Specify path to image file containing wingle object on white background
input_file_path = './test-images/test1.jpg'
# Specify path for writing output file
output_file_path = os.path.join('./output/', f'{os.path.splitext(os.path.split(input_file_path)[-1])[0]}.png')

## Functions Definition

In [ ]:
def get_contours(img, j_thres, s_thres):
    '''Create contours based on edge detection and morphological operations

    j_thres:
    Join threshold for adjusting the integration level of individual characters / object. 
    Increase this value if the tool fails to segment large characters / objects or if the drawing lines is too faint. 
    The valid value is 1 to 15.
    s_thres
    Separation threshold for adjusting the separation level of neighboring characters / object. 
    Increase this value if the tool fails to segment characters / objects that is close one to another. 
    The valid value is 1 to 15.
    '''
    # Create grayscale version
    img_gray = cv.cvtColor(img.copy(), cv.COLOR_BGR2GRAY)
    # Noise removal
    img_gray = cv.GaussianBlur(img_gray, (7,7), 0)
    # Edge detection
    edge = cv.Canny(img_gray, 30, 70)
    # Dilate for getting sure foreground (object) area
    edge = cv.dilate(edge, np.ones((j_thres,j_thres)), 1)
    # Closing black holes inside the object with area < 10000 square pixels
    edge = morph.area_closing(edge, 10000)

    #cv.imwrite('./output/edge.png', edge)
    # Try to separate touching objects
    edge = cv.erode(edge, np.ones((s_thres,s_thres)), 1)
    # Find contour
    cts,_ = cv.findContours(edge.astype('uint8'), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    return cts


def get_large_contours(cts, size):
    '''Return contouts larger than size'''
    area = []
    large_contour = []
    for ct in cts:
        a = cv.contourArea(ct)
        if a > size:
            area.append(a)
            large_contour.append(ct)
    return area, large_contour


def draw_bboxes(img, contours, line_color, line_width):
    '''Create sheet images with object bounding boxes'''

    def __create_bboxes(contours):
        '''Create oject bounding boxes from contours'''
        bboxes = []
        for ct in contours:    
            x,y,w,h = cv.boundingRect(ct)
            bboxes.append([x,y,w,h])
        return bboxes

    def __get_bbox_points(bbox, img_shape, pad_factor=0):
        '''Return bounding box corner points'''
        x, y, w, h = bbox
        # Padding
        y1 = y - int(h*pad_factor)
        y2 = y + h + int(h*pad_factor)
        x1 = x - int(w*pad_factor)
        x2 = x + w + int(w*pad_factor)
        # Limiting
        if y1 < 0:
            y1 = 0
        if y2 > img_shape[0]-1:
            y2 = img_shape[0]-1
        if x1 < 0:
            x1 = 0
        if x2 > img_shape[1]-1:
            x2 = img_shape[1]-1
        return x1, x2, y1, y2

    print ('Creating bounding boxes embedded image...')
    # Creating rectangles to bound objects
    bboxes = __create_bboxes(contours)
    # Create a copy for drawing bboxes
    img_bbox = img.copy()
    # Writing to file
    i = 1
    for bbox in bboxes:
        #print (f'Drawing box {i} from {len(bboxes)}')
        x1, x2, y1, y2 = __get_bbox_points(bbox, img_bbox.shape)
        cv.rectangle(img_bbox,(x1,y1),(x2,y2), line_color, line_width)
        i+=1
    return img_bbox

## Detect Object and Draw Bounding Box

In [ ]:
# Reading image
img = cv.imread(input_file_path, 1)
'''
Setting the join threshold (j_thres) and separation threshold (s_thres)
- j_thres:
Join threshold for adjusting the integration level of individual characters / object. 
Increase this value if the tool fails to segment large characters / objects or if the drawing lines is too faint. 
The valid value is 1 to 15.
- s_thres
Separation threshold for adjusting the separation level of neighboring characters / object. 
Increase this value if the tool fails to segment characters / objects that is close one to another. 
The valid value is 1 to 15.
'''
j_thres = 10
s_thres = 1

# Mark start time
t_start = time.time()

# Get object contours
cts = get_contours(img, j_thres, s_thres)
# Get contour with area larger than 300 pixel square. The smaller one considered noise
_, lcts = get_large_contours(cts, size=1000)
# Tell the number of objects found
n_objects = len(lcts)
print (f'Objects found: {n_objects}')
img_bbox = draw_bboxes(img, lcts, line_color = (0,255,0), line_width=5)

# Mark completed time
t_end = time.time()

print (f'Processing time: {t_end-t_start}')
# Writing to file
cv.imwrite(output_file_path, img_bbox)
print (f'Bounding box image created as {output_file_path}')